In [126]:
# Bybitが提供している公式のライブラリを用いる
# 使い方は以下のURL参照
# https://bybit-exchange.github.io/docs/linear/#t-constructingtherequest
import bybit
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import pandas as pd

%matplotlib inline

In [127]:
# Bybit API Key (test net)
# Test API は3ヶ月の有効期限を過ぎると使えなくなる(再発行可能)
api_key = "your API"
api_secret = "your Secret key"


# クラスのインスタンス化
# Bybitにはdemo用のtest環境と取引用の本番環境の2種類があり、それぞれでAPI_keyが異なる
# test = Trueにすることでdemo用環境を指定できる

client = bybit.bybit(test=True, api_key = api_key, api_secret = api_secret)

In [128]:
# ローソク足データを取得する 取得できるのは一度に最大200件まで
# 取得したいデータの期間 ここで指定した日時から最大200件のローソク足データが手に入る
date_str = '2021/9/4 00:00'
date_dt = datetime.strptime(date_str, '%Y/%m/%d %H:%M')
target_unix = date_dt.timestamp()
print(target_unix)

1630681200.0


In [129]:
# ローソク足の時間足はintervalで指定 60 = 1時間足
# 1時間足以外に1分足から日足 月足 色々ある　詳細は公式github参照
# 取得したいデータの期間をunix時間で指定しないといけないことがやや面倒

Historical_data = client.Kline.Kline_get(
    symbol="BTCUSD",interval="60", **{'from': target_unix}).result()

# 返り値のparameter
id: 多分データ管理用に割り振られた番号　そんな重要じゃなさそう
'ret_code': 0, 'ret_msg': 'OK', 'ext_code': '', 'ext_info これらはAPIリクエストが成功したかどうかのメッセージ

データはresult以降に格納されている
以下 result内部のparameterについて

symbol: 取引ペア

period: 時間足

start_at, open_time: 時間 unix timeで入ってる startとopenでどう違うのかは不明

open,high,low,close: 始値,高値,安値,終値

volume: 取引数量

turnover: ? 売り上げという意味らしいがvolumeと何が違うのか不明

time_now:　プログラムを実行したときのunix時間

In [130]:
# resultだけを取ってくる
Data = Historical_data[0]['result']

In [131]:
Date = []
Close_price = []
Open_price = []
High_price = []
Low_price = []
Volume = []
for i in range(len(Data)):
    Date.append(datetime.fromtimestamp(Data[i]['open_time']))
    Close_price.append(Data[i]['close'])
    Open_price.append(Data[i]['open'])
    High_price.append(Data[i]['high'])
    Low_price.append(Data[i]['low'])
    Volume.append(Data[i]['volume'])

    
# pandas dataframeを作成
df = pd.DataFrame({     'date' : Date,
                        'open' : Open_price,
                        'high' : High_price,
                        'low' : Low_price,
                        'close' : Close_price,
                        'volume' : Volume })

# df = df.set_index('date')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    200 non-null    datetime64[ns]
 1   open    200 non-null    object        
 2   high    200 non-null    object        
 3   low     200 non-null    object        
 4   close   200 non-null    object        
 5   volume  200 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 9.5+ KB


In [132]:
# plt.plot(df.index, df['close'])
# plt.xticks(rotation=30)
# plt.xlabel('Date')
# plt.ylabel('BTC price (USDT)')
# plt.title('BTC price change in Bybit')

In [133]:
# ローソク足の表示 なんか色々動かせて超便利

fig = go.Figure(data=[go.Candlestick(
                        x=df["date"],
                        open=df['open'],
                        high=df['high'],
                        low=df['low'],
                        close=df['close'],
                        increasing_line_color= 'darkgreen',
                        decreasing_line_color= 'red')])

fig.update_layout(autosize=True,width=1000,height=800)
fig

## Liquidation 清算データ
最大で1000件取得できる
start time, end time で取得したい期間を指定できる

返り値の qty が清算注文量を表す単位はドル


In [134]:
Liquidation = client.Market.Market_liqRecords(symbol="BTCUSD", limit=1000).result()

print(len(Liquidation[0]["result"]))

452


In [135]:
Liquidation_data = Liquidation[0]["result"]

Date_liquid_buy = []
Date_liquid_sell = []
qty_buy = []
qty_sell = []
price_buy = []
price_sell = []

for i in range(len(Liquidation_data)):
    if Liquidation_data[i]["side"] == "Buy":
        Date_liquid_buy.append(datetime.fromtimestamp(Liquidation_data[i]['time']/1000.0))
        qty_buy.append(Liquidation_data[i]['qty'])
        price_buy.append(Liquidation_data[i]['price'])
    else:
        Date_liquid_sell.append(datetime.fromtimestamp(Liquidation_data[i]['time']/1000.0))
        qty_sell.append(Liquidation_data[i]['qty'])
        price_sell.append(Liquidation_data[i]['price'])
    

In [136]:
print(Liquidation_data[0]['time'])
print(datetime.fromtimestamp(Liquidation_data[0]['time']/1000.0))

1630869210490
2021-09-06 04:13:30.490000


In [137]:
df_liquid_buy = pd.DataFrame({'date' : Date_liquid_buy,
                        'qty_buy' : qty_buy,
                        'price_buy' : price_buy,
                        })

df_liquid_sell = pd.DataFrame({'date' : Date_liquid_sell,
                        'qty_sell' : qty_sell,
                        'price_sell' : price_sell,
                        })


#df_liquid = df_liquid.set_index('date')
df_liquid_buy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       293 non-null    datetime64[ns]
 1   qty_buy    293 non-null    int64         
 2   price_buy  293 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 7.0 KB


In [138]:
# Groupby methodでdate columnが indexになってしまうのでreset_indexで対処

df_liquid_buy = df_liquid_buy.groupby(pd.Grouper(key='date', freq='1h')).sum()
df_liquid_buy.reset_index(inplace=True)
df_liquid_sell = df_liquid_sell.groupby(pd.Grouper(key='date', freq='1h')).sum()
df_liquid_sell.reset_index(inplace=True)
df_liquid_sell.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        163 non-null    datetime64[ns]
 1   qty_sell    163 non-null    int64         
 2   price_sell  163 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 3.9 KB


In [139]:
df_liquid_buy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       144 non-null    datetime64[ns]
 1   qty_buy    144 non-null    int64         
 2   price_buy  144 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 3.5 KB


## Open Interest  未約定ポジション数
最大200件取得可能
なぜかこれだけ古-新でなく新-古の順でデータが返ってくる

In [140]:
OpenInterest = client.Market.Market_openInterest(
    symbol="BTCUSD", limit=200, period="1h"
).result()

In [141]:
OpenInterest_data = OpenInterest[0]["result"]

print(len(OpenInterest_data))

200


In [142]:
Date_open = []
open_interest = []

for i in range(len(Data)):
    Date_open.append(datetime.fromtimestamp(OpenInterest_data[i]['timestamp']))
    open_interest.append(OpenInterest_data[i]['open_interest'])
    

In [143]:
df_open = pd.DataFrame({     'date' : Date_open[::-1],
                        'open_interest' : open_interest[::-1],
                        })

#df_open = df_open.set_index('date')
df_open.head()

,date,open_interest
0,2021-09-04 18:00:00,219060969
1,2021-09-04 19:00:00,216284855
2,2021-09-04 20:00:00,228145776
3,2021-09-04 21:00:00,229712569
4,2021-09-04 22:00:00,232732534


OHLCV, OpenInterest, Liquidationデータはそれぞれ取得できる限度が異なるため同期間のデータを手に入れるにはどれかにデータの範囲を合わせる必要がある
OHLCV, OpenInerestは1時間ごとのデータを手に入れられるがLiquidationデータは日時がバラバラで3つの指標をプロットしてデータ分析するためにはデータを1時間ごとにまとめ直す必要がある

## OHLCV OpenInterest Liquidation dataを一つのDataFrameにまとめる

In [144]:
# OHLCVとOpenInterest Dataを結合する
# OHLCV dataが9/1~ OpenInterestが9/4~ なので結合データは9/4~になる

df_1 = pd.merge(df, df_open)

df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 0 to 181
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           182 non-null    datetime64[ns]
 1   open           182 non-null    object        
 2   high           182 non-null    object        
 3   low            182 non-null    object        
 4   close          182 non-null    object        
 5   volume         182 non-null    object        
 6   open_interest  182 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 11.4+ KB


In [145]:
# OpenInetestと同時刻のclose priceを同時にプロット

fig = go.Figure(
    data = [
        go.Bar(x = df_1["date"], y = df_1['open_interest'], name = 'OpenInterest',yaxis='y1'),
        go.Scatter(x = df_1["date"], y = df_1['close'] , name = 'ClosePrice',  yaxis='y2')
    ],
    layout = go.Layout(
        xaxis = dict(showgrid=False, tickangle=-60),
        yaxis = dict(title = 'OpenInterest', side = 'right', showgrid=False),
        yaxis2 = dict(title = 'ClosePrice', side = 'left',showgrid=False, overlaying = 'y'),
        margin=dict(l=20, r=10, t=20, b=100, autoexpand=True),
        legend=dict(x=1.05, xanchor='left', y=1, yanchor='auto'),
        height=400,
        width=1000)
)
fig.update_layout(barmode='group')
fig.show()

## メモ

Open Interestが急減した直後あたりに終値が急変しているところがいくつかある
Open Interestの急減と価格の方向性には関係がなさそう
9/8の急落前後ではOpen Interest量に大きな差がある
Open Interestの数量変化は取引指標として使えるかも？

In [146]:
# buy sell 別々にしたliquidation dataを一つにまとめる
df_liquid = pd.merge(df_liquid_buy, df_liquid_sell, how="outer")

In [147]:
df_liquid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163 entries, 0 to 162
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        163 non-null    datetime64[ns]
 1   qty_buy     144 non-null    float64       
 2   price_buy   144 non-null    float64       
 3   qty_sell    163 non-null    int64         
 4   price_sell  163 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 7.6 KB


In [148]:
df_liquid.head()

,date,qty_buy,price_buy,qty_sell,price_sell
0,2021-09-06 20:00:00,123.0,51201.5,5952299,418757.0
1,2021-09-06 21:00:00,1.0,50968.5,0,0.0
2,2021-09-06 22:00:00,847.0,50963.0,0,0.0
3,2021-09-06 23:00:00,0.0,0.0,12100,155845.0
4,2021-09-07 00:00:00,0.0,0.0,0,0.0


In [149]:
# OHLCV OpenInterest Liquidation を一つのデータとしてdataframeにまとめる

df_all = pd.merge(df_1, df_liquid)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148 entries, 0 to 147
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           148 non-null    datetime64[ns]
 1   open           148 non-null    object        
 2   high           148 non-null    object        
 3   low            148 non-null    object        
 4   close          148 non-null    object        
 5   volume         148 non-null    object        
 6   open_interest  148 non-null    int64         
 7   qty_buy        132 non-null    float64       
 8   price_buy      132 non-null    float64       
 9   qty_sell       148 non-null    int64         
 10  price_sell     148 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 13.9+ KB


In [150]:
df_all.head()

,date,open,high,low,close,volume,open_interest,qty_buy,price_buy,qty_sell,price_sell
0,2021-09-06 04:00:00,50422.5,50963,50401.5,50701.5,68699659,252947034,NaN,NaN,12316078,305294.0
1,2021-09-06 05:00:00,50701.5,51800,50699,51727.5,95932496,245196011,NaN,NaN,32838181,1699837.0
2,2021-09-06 06:00:00,51727.5,52004,51559,51786.5,40664240,211740274,NaN,NaN,3082166,260540.0
3,2021-09-06 07:00:00,51786.5,51947.5,51671.5,51874,38604999,210863088,NaN,NaN,0,0.0
4,2021-09-06 08:00:00,51874,51881.5,51705.5,51819,28166727,226988983,NaN,NaN,0,0.0


9/6 4:00 からのデータを取得することができた 約1週間分
ここからbuy sell 両サイドのLiquidationの数量も含めた分析をしていく

In [159]:
# close が数値型になっていないので変換
df_all["close"] = df_all["close"].astype(float)

# close と open interestの変化率を列に追加
df_all["close_change"] = df_all["close"].pct_change()
df_all["open_interest_change"] = df_all["open_interest"].pct_change()

# 欠損値を0で埋める
df_all = df_all.fillna(0)

In [160]:
df_all.head()

,date,open,high,low,close,volume,open_interest,qty_buy,price_buy,qty_sell,price_sell,close_change,open_interest_change
0,2021-09-06 04:00:00,50422.5,50963,50401.5,50701.5,68699659,252947034,0.0,0.0,12316078,305294.0,0.000000,0.000000
1,2021-09-06 05:00:00,50701.5,51800,50699,51727.5,95932496,245196011,0.0,0.0,32838181,1699837.0,0.020236,-0.030643
2,2021-09-06 06:00:00,51727.5,52004,51559,51786.5,40664240,211740274,0.0,0.0,3082166,260540.0,0.001141,-0.136445
3,2021-09-06 07:00:00,51786.5,51947.5,51671.5,51874.0,38604999,210863088,0.0,0.0,0,0.0,0.001690,-0.004143
4,2021-09-06 08:00:00,51874,51881.5,51705.5,51819.0,28166727,226988983,0.0,0.0,0,0.0,-0.001060,0.076476
